## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [638]:
import logging
import pandas as pd
import numpy as np
import geopy.distance
import networkx as nx
import random

from itertools import combinations
from dataclasses import dataclass
from icecream import ic
from tqdm.auto import tqdm

In [639]:

countries = ['China', 'Italy', 'Russia', 'US', 'Vanuatu']
cities = []
dist_matrix = []


## Helper functions

In [640]:

@dataclass
class Individual:
    genome: np.ndarray
    fitness: float = None


def tsp_cost(tsp):
    #VALIDATING the tsp array
    #Ensures that the path starts and ends at the same city 
    assert tsp[0] == tsp[-1]
    #Checks that the path includes all cities exactly once before returning to the start
    assert set(tsp) == set(range(len(cities)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost


def fitness(path):
    return float(tsp_cost(path))

## Loading data

In [641]:

def init_data(selected_country):
    # Loading data
    cities = pd.read_csv(f'cities/{selected_country}.csv', header=None, names=['name', 'lat', 'lon'])

    # Calculating distances between cities
    dist_matrix = np.zeros((len(cities), len(cities)))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km

    return cities, dist_matrix


## Greedy Algorithm

In [642]:
def greedy_tsp(cities, dist_matrix, init_city):
    visited = np.full(len(cities), False)
    dist = dist_matrix.copy()
    visited[init_city] = True
    tsp = list()
    tsp.append(int(init_city))
    while not np.all(visited):
        dist[:, init_city] = np.inf
        closest = np.argmin(dist[init_city])
        visited[closest] = True
        init_city = closest
        tsp.append(int(init_city))
      
    tsp.append(tsp[0])
    return tsp

## Genetic Algorithm + Greedy Algorithm

#### General Settings for each instance

In [643]:
BEST_RESULTS = ['-', '4172.76', '32722.5', '3899745', '1345.54']
PERC_GREEDYs = [0.2, 0.4, 0.2, 0.2, 0.2]  
MAX_GENERATIONS_list = [1000, 1000, 1000, 1000, 100]  
POPULATION_SIZES = [500, 300, 500, 500, 200] 
OFFSPRING_SIZES = [300, 200, 300, 300, 100]  

In [644]:
# PARENT SELECTION
# Fitness proportional
def parent_selection(population):
    candidates = sorted(np.random.choice(population, 2), key=lambda e: e.fitness, reverse=True)
    return candidates[0]


# ************ CROSSOVER
def inverOver(p1: Individual, p2: Individual, init_city=0):
        new_path = []
        # Extract the paths without start and end city
        path1 = p1.genome[1:-1]
        path2 = p2.genome[1:-1]
        
        # Select a random element in path1
        first_index_p1 = np.random.randint(len(path1))
        first_city = path1[first_index_p1]

        # Find the selected element in path2 and determine the second element
        first_index_p2 = path2.index(first_city)
        second_city = path2[first_index_p2 + 1] if first_index_p2 + 1 < len(path2) else path2[0]

        # Find the position of the second element in path1
        second_index_p1 = path1.index(second_city)

        # Perform inversion based on relative positions of indices
        if second_index_p1 > first_index_p1:
            traits_p1 = path1[first_index_p1 + 1:second_index_p1][::-1]
            new_path += path1[:first_index_p1] + [first_city, second_city] + traits_p1 + path1[second_index_p1 + 1:]
        else:
            traits_p1 = path1[second_index_p1 + 1:first_index_p1][::-1]
            new_path = path1[:second_index_p1] + [second_city, first_city] + traits_p1 + path1[first_index_p1 + 1:]

        # Adding init_city at the start and end
        new_path = [init_city] + new_path + [init_city]
       
        return Individual(new_path, fitness(new_path))
        


#*************** POSSIBLE MUTATIONS
# SWAP MUTATION
def swap_mutation(p: Individual,init_city=0, prob = None):
    genome = p.genome.copy()

    while np.random.random() < 0.2:
        #find 2 indexes (ignoring start and end city)
        i, j = np.random.choice(np.arange(1, len(genome) - 1), size=2, replace=False)
        # Swapping 2 elements
        genome[i], genome[j] = genome[j], genome[i]

    return Individual(genome, fitness(genome))



# SCRUMBLE MUTATION
def scrumble_mutation(p: Individual, init_city=0, prob = None):
    genome = p.genome.copy()
    
    # Take a random number of cities to change (from 2 changes to half of possible cities)
    num_loci = np.random.randint(2, int((len(genome) - 2) / 2))
    
    # Select random positions without start and end city
    positions = np.random.choice(np.arange(1, len(genome) - 2), size=num_loci, replace=False)
    # Selected cities
    selected_cities = [genome[i] for i in positions]

    # Shuffle the selected cities
    random.shuffle(selected_cities)
    
    # Assign the shuffled cities to the selected positions
    for idx, city in zip(positions, selected_cities):
        genome[idx] = city

    return Individual(genome, fitness(genome))


# INVERSION MUTATION
def inversion_mutation(p: Individual, init_city=0, prob = None):
    # Extract the paths without start and end city
    path = p.genome[1:-1]
    n = len(path)

    # Choosing 2 random indexes to perform the inversion
    first, second = sorted(np.random.choice(n, 2, replace=False))
    
    # Creating a new route by reversing the segment between first and second
    mutated_path =  path[:first] + path[first:second + 1][::-1] + path[second + 1:]
    # Adding start and end city
    new_path =  [init_city] + mutated_path + [init_city]

    return Individual(new_path, fitness(new_path))


In [645]:
# Generate a random individual
def generate_random_individual(init_city=0):
    num_cities = len(cities)
    # Randomly select cities, excluding init_city, avoiding duplicates
    path = [init_city] + list(np.random.choice([i for i in range(num_cities) if i != init_city], size=num_cities - 1, replace=False)) + [init_city]
    return Individual(path, fitness(path))


# Create initial population
def fill_population(percentage, size, init_solution, init_city=0):
    # Number of initial solutions to start from
    num_init_solution = int(percentage * size)
    
    # Adding individuals with initial solutions as genome
    population = [Individual(init_solution, fitness(init_solution)) for _ in range(num_init_solution)]
    
    # Adding random individuals
    population += [generate_random_individual(init_city) for _ in range(size - num_init_solution)]

    return population


In [646]:
def genetic_algorithm(init_city,init_solution, percentage_greedy, POPULATION_SIZE, OFFSPRING_SIZE, MAX_GENERATIONS, mutation_type, xover_type):
    
    # Initialize population with all equal greedy solution
    population = fill_population(percentage_greedy, POPULATION_SIZE, init_solution, init_city)

    mutation = mutation_type
    xover = xover_type

    # Initial probability to apply a mutation
    strength = .3

    for g in tqdm(range(MAX_GENERATIONS)):
        offspring = list()
        
        # Self adaptive strength -> we prefer mutation (exploitation) going on with the generations
        if g % 100 == 0 and g != 0:
            strength += 0.2

        for _ in range(OFFSPRING_SIZE):
            #HYPERMODERN
            if np.random.random() < strength:
                #MUTATION
                p = parent_selection(population)
                o = mutation(p, init_city)
            else:
                #RECOMBINATION
                p1 = parent_selection(population)
                p2 = parent_selection(population)
                o = xover(p1, p2, init_city)

            offspring.append(o)

        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=False)

        population = population[:POPULATION_SIZE]

    return population[0].genome, population[0].fitness


In [648]:
#Choose the country
selected_country = 0 #China
init_city = 0


print('-----------------------------')
print(f"\nProcessing: {countries[selected_country]}")
        
# Loading city data and generate distance matrix
cities, dist_matrix = init_data(countries[selected_country])
print(f"Number of cities: {dist_matrix.shape[0]}")
        
# Generating a greedy initial solution
greedy_solution = greedy_tsp(cities, dist_matrix, init_city)
print(f"Initial distance: {fitness(greedy_solution)} km")

# Genetic algorithm to find the best solution
best_path, best_distance = genetic_algorithm(
            init_city, greedy_solution, 
            percentage_greedy=PERC_GREEDYs[selected_country], 
            POPULATION_SIZE=POPULATION_SIZES[selected_country], 
            OFFSPRING_SIZE=OFFSPRING_SIZES[selected_country], 
            MAX_GENERATIONS=MAX_GENERATIONS_list[selected_country],
            mutation_type=inversion_mutation, 
            xover_type=inverOver
        )

# Results
print(f"\nBest distance found: {best_distance} km")
print(f"Number of steps: {len(best_path) - 1}")
print(f"\nKnown best result: {BEST_RESULTS[selected_country]} km")

-----------------------------

Processing: China
Number of cities: 726
Initial distance: 63962.918429455196 km


  0%|          | 0/1000 [00:00<?, ?it/s]


Best distance found: 62826.3790916235 km
Number of steps: 726

Known best result: - km
